# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Install required dependencies
import sys
!{sys.executable} -m pip install "chromadb>=1.0.4" "openai>=1.73.0" "pydantic>=2.11.3" "python-dotenv>=1.1.0" "tavily-python>=0.5.4" "pdfplumber" "tavily-python"


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


In [2]:
import os
from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from dotenv import load_dotenv
import json
import chromadb
from tavily import TavilyClient
import time

In [3]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("BASE_URL")
tavily_api_key = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [4]:
@tool
def retrieve_game(query: str) -> list:
  """
    Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry.

    You'll receive results as list. Each element contains:
    - Name: Name of the Game
    - Platform: like Game Boy, Playstation 5, Xbox 360...
    - Genre: like Action, Adventure, RPG...
    - Publisher: Company that published the game
    - Description: Additional details about the game
    - YearOfRelease: Year when that game was released for that platform

  """
  chroma_client = chromadb.PersistentClient(path="chromadb")
  collection = chroma_client.get_collection("udaplay")
  result = collection.query(query_texts=[query], n_results=5)
  formated_result = []
  result_lenght = len(result["documents"][0])
  for i in range(result_lenght):
    formated_result.append(f"{i+1}. {result['documents'][0][i]}")
  return "\n".join(formated_result)

#### Evaluate Retrieval Tool

In [5]:
@tool
def evaluate_retrieval(question: str, retrieved_docs: str) -> str:
  """
    Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.
    args:
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database

    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
  """
  llm = LLM(model="gpt-3.5-turbo", openai_api_key=openai_api_key, base_url=base_url)
  system_message = SystemMessage(content="You are an expert AI assistant that helps people find information.")
  user_message = UserMessage(content=f"Your task is to evaluate if the documents are enough to respond the query.\n"
  f"Question: {question}\n"
  f"Documents: {retrieved_docs}\n"
  f"Give a detailed explanation, so it's possible to take an action to accept it or not.")

  messages = [system_message, user_message]
  response = llm.invoke(messages)
  return response.content

#### Game Web Search Tool

In [6]:
@tool
def game_web_search(question: str) -> str:
  """
    Search the web using Tavily to find relevant information about the game industry.
    args:
    - question: a question about game industry.

    The result includes:
    - A summary of the most relevant information found on the web.
  """
  from tavily import TavilyClient

  tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
  search_results = tavily.search(query=question, max_results=5)

  if not search_results or 'results' not in search_results:
    return "No results found."

  results = search_results['results']
  formatted_results = []
  for i, result in enumerate(results):
    title = result.get('title', 'No Title')
    snippet = result.get('snippet', 'No Snippet')
    url = result.get('url', 'No URL')
    formatted_results.append(f"{i+1}. {title}\n{snippet}\n{url}\n")

  return "\n".join(formatted_results)

### Agent

In [7]:
tools = [retrieve_game, evaluate_retrieval, game_web_search]

agent_instructions = (
    "You are UdaPlay, an AI research assistant specializing in video games.\n"
    "Maintain conversation context and remember previous questions and answers.\n\n"

    "MANDATORY WORKFLOW - YOU MUST FOLLOW THIS EXACT SEQUENCE:\n"
    "1. ALWAYS start by calling retrieve_game to search internal knowledge\n"
    "2. ALWAYS call evaluate_retrieval to assess the quality of retrieved information\n"
    "3. ONLY IF evaluation indicates insufficient information, call game_web_search\n"
    "4. Provide clear, structured answers with proper source citations\n"
    "5. Reference previous conversation context when relevant\n\n"

    "RESPONSE FORMATTING:\n"
    "- Always structure your final answer clearly\n"
    "- Include source citations (e.g., '[Source: Internal Database]' or '[Source: Web Search - URL]')\n"
    "- When referencing previous conversation, explicitly mention it\n"
    "- Synthesize information from multiple sources when available\n\n"
)

agent = Agent(
    model_name="gpt-4o-mini",
    tools=tools,
    openai_api_key=openai_api_key,
    base_url=base_url,
    instructions=agent_instructions
)

In [8]:
def run_query_with_detailed_logging(query: str, session_id: str):
    """
    Run a query with detailed logging to verify tool execution sequence.
    """

    print("Run a query with detailed logging to verify tool execution sequence: ")
    print(f"Query: {query}")
    print(f"Session ID: {session_id}")

    start_time = time.time()

    tool_executions = []

    class LoggingAgent(Agent):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.tool_history = []

        def _execute_tool(self, tool_name, tool_args):
            print(f"\n🔧 Executing tool: {tool_name}")
            print(f"   Arguments: {tool_args}")
            result = super()._execute_tool(tool_name, tool_args)

            result_preview = str(result)[:200] if result else "No result"
            if len(str(result)) > 200:
                result_preview += "..."
            print(f"   Result preview: {result_preview}")

            tool_executions.append({
                'name': tool_name,
                'args': tool_args,
                'result': result
            })

            return result

    try:
        response = agent.invoke(query=query, session_id=session_id)
    except Exception as e:
        print(f"❌ Error during agent execution: {e}")
        response = None

    execution_time = time.time() - start_time

    print(f"  📨 Session ID: {session_id}")

    print(f"\n💬 Final Answer:")

    if response:
        final_state = response.get_final_state()
        if final_state and "messages" in final_state:
            # Get the last AI message
            ai_messages = [msg for msg in final_state["messages"] if isinstance(msg, AIMessage)]
            if ai_messages:
              actual_answer = ai_messages[-1].content
              print(actual_answer)
            else:
                print("❌ No AI response found in messages")
        else:
            print("❌ No final state or messages found")
    else:
        print("❌ No response received from agent")


    return response

In [9]:
def extract_agent_response(run_object):
    """
    Helper function to extract the actual text response from an agent's Run object
    """
    if not run_object:
        return "No response object"

    final_state = run_object.get_final_state()
    if final_state and "messages" in final_state:
        ai_messages = [msg for msg in final_state["messages"] if isinstance(msg, AIMessage)]
        if ai_messages:
            return ai_messages[-1].content
        else:
            return "No AI response found in messages"
    else:
        return "No final state or messages found"

def run_query_with_state_tracking(query: str, session_id: str, agent: Agent):
    """
    Enhanced query execution with detailed state and tool tracking.
    """
    print(f"🎮 UdaPlay Query Processor")
    print(f"📝 Query: {query}")
    print(f"🔑 Session: {session_id}")

    start_time = time.time()

    print(f"\n📍 Executing agent workflow...")

    try:
        response = agent.invoke(query=query, session_id=session_id)

        execution_time = time.time() - start_time
        print(f"📊 Execution Time: {execution_time:.2f}s")
        print(f"✅ Status: Success")

        print(f"\n🎯 Agent Response:")

        final_answer = extract_agent_response(response)
        print(final_answer)

        print("-"*50)
        return response

    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        return None

In [10]:
questions = [
    "When was Pokémon Gold and Silver released?",
    "What platforms was it available on?",  # This question references the previous one
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for PlayStation 5?"
]

session_id = "conversation_1"

print(f"🎮 Starting UdaPlay conversation session | SessionId: {session_id}")
print(f"Testing {len(questions)} queries with conversation continuity...\n")

responses = []
for i, question in enumerate(questions, 1):
    print(f"# Question {i}/{len(questions)}")
    response = run_query_with_detailed_logging(question, session_id=session_id)
    actual_answer = extract_agent_response(response)
    print(f"# Answer {i}: {actual_answer}")

    responses.append(response)

    if i < len(questions):
        print(f"\n{'.'*70}")
        print("Moving to next question...")
        print(f"{'.'*70}")

print(f"🏁 Conversation session completed!")
print(f"Session '{session_id}' processed {len(questions)} queries with maintained context.")

🎮 Starting UdaPlay conversation session | SessionId: conversation_1
Testing 4 queries with conversation continuity...

# Question 1/4
Run a query with detailed logging to verify tool execution sequence: 
Query: When was Pokémon Gold and Silver released?
Session ID: conversation_1
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
  📨 Session ID: conversation_1

💬 Final Answer:
Pokémon Gold and Silver were released on **October 15, 2000**, in North America for the Game Boy Color. These games marked the second generation of Pokémon and introduced new regions, Pokémon, and gameplay mechanics. 

If you have an

### Conversation Context Test

This cell demonstrates that the agent maintains conversation state across multiple queries in the same session:

In [11]:
# Test conversation context with related questions
context_test_questions = [
    "Tell me about Super Mario 64",
    "When was it released?",  # "it" should refer to Super Mario 64 from previous question
    "What made this game revolutionary?"  # "this game" should refer to Super Mario 64
]

session_id_context = "context_test_session"

print(f"🧪 Testing conversation context maintenance")
print(f"Session ID: {session_id_context}")
print(f"{'='*70}\n")

for i, question in enumerate(context_test_questions, 1):
    print(f"\n{'='*70}")
    print(f"Question {i}: {question}")
    print(f"{'='*70}")

    response = run_query_with_state_tracking(question, session_id_context, agent)

    if i < len(context_test_questions):
        print(f"\n{'.'*50}")

print(f"\n{'='*70}")
print(f"✅ Context test completed")
print(f"The agent should have maintained context across all {len(context_test_questions)} questions")
print(f"Questions 2 and 3 relied on context from Question 1")
print(f"{'='*70}")

🧪 Testing conversation context maintenance
Session ID: context_test_session


Question 1: Tell me about Super Mario 64
🎮 UdaPlay Query Processor
📝 Query: Tell me about Super Mario 64
🔑 Session: context_test_session

📍 Executing agent workflow...
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
📊 Execution Time: 22.28s
✅ Status: Success

🎯 Agent Response:
### Overview of Super Mario 64

**Release Information:**
- **Platform:** Nintendo 64
- **Release Year:** 1996

**Description:**
Super Mario 64 is a landmark 3D platformer that revolutionized the gaming industry. In the game, players control Mario as he e

In [12]:

def test_agent_responses():
    """Test the agent and display responses clearly"""

    test_questions = [
        "When was Pokémon Gold and Silver released?",
        "Tell me about Super Mario 64",
        "What platforms was it available on?",  # Should refer to Mario 64
    ]

    session_id = "demo_session"
    print("🎮 UdaPlay Agent Demo - Showing Agent Responses")

    for i, question in enumerate(test_questions, 1):
        print(f"QUESTION {i}: {question}")

        try:
            response = agent.invoke(query=question, session_id=session_id)

            final_state = response.get_final_state()
            if final_state and "messages" in final_state:
                ai_messages = [msg for msg in final_state["messages"] if isinstance(msg, AIMessage)]
                if ai_messages:
                    answer = ai_messages[-1].content
                else:
                    answer = "No AI response found"
            else:
                answer = "No final state found"

            print(f"\n🤖 AGENT ANSWER:")
            print(answer)


        except Exception as e:
            print(f"❌ Error: {str(e)}")

test_agent_responses()

🎮 UdaPlay Agent Demo - Showing Agent Responses
QUESTION 1: When was Pokémon Gold and Silver released?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

🤖 AGENT ANSWER:
Pokémon Gold and Silver were released in 1999 for the Game Boy Color. These games are notable for introducing new regions, Pokémon, and gameplay mechanics as part of the second generation of Pokémon games. 

If you have any more questions about Pokémon or other games, feel free to ask! [Source: Internal Database]
QUESTION 2: Tell me about Super Mario 64
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMa

### (Optional) Advanced

In [13]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes

### Tool Testing


Test individually to ensure they work correctly:

In [14]:
# Test 1: retrieve_game tool
print("Testing retrieve_game tool:")
print("-" * 50)
try:
    result = retrieve_game("Pokemon Gold and Silver")
    print(f"✅ retrieve_game executed successfully")
    print(f"Results:\n{result[:500]}...")  # Show first 500 chars
except Exception as e:
    print(f"❌ retrieve_game failed: {e}")

print("\n" + "="*60 + "\n")

Testing retrieve_game tool:
--------------------------------------------------
✅ retrieve_game executed successfully
Results:
1. [Game Boy Color] Pokémon Gold and Silver (1999) - Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.
2. [Game Boy Advance] Pokémon Ruby and Sapphire (2002) - Third-generation Pokémon games set in the Hoenn region, featuring new Pokémon and double battles.
3. [Super Nintendo Entertainment System (SNES)] Super Mario World (1990) - A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser.
4. [Nintendo 64] ...




In [15]:
# Test 2: evaluate_retrieval tool
print("Testing evaluate_retrieval tool:")
print("-" * 50)
try:
    # Use results from previous test
    evaluation = evaluate_retrieval(
        question="When was Pokemon Gold and Silver released?",
        retrieved_docs=result if 'result' in locals() else "Sample game data"
    )
    print(f"✅ evaluate_retrieval executed successfully")
    print(f"Evaluation:\n{evaluation}")
except Exception as e:
    print(f"❌ evaluate_retrieval failed: {e}")

print("\n" + "="*60 + "\n")

Testing evaluate_retrieval tool:
--------------------------------------------------
✅ evaluate_retrieval executed successfully
Evaluation:
Based on the provided documents, the information about Pokémon Gold and Silver is present in the first document. The document states that Pokémon Gold and Silver were second-generation Pokémon games released in 1999. This information directly answers the query about the release date of Pokémon Gold and Silver.

Therefore, the documents are sufficient to respond to the query about when Pokémon Gold and Silver was released. The information provided in the first document clearly states that Pokémon Gold and Silver was released in 1999.




In [16]:
# Test 3: game_web_search tool
print("Testing game_web_search tool:")
print("-" * 50)
try:
    web_result = game_web_search("latest Pokemon game release 2024")
    print(f"✅ game_web_search executed successfully")
    print(f"Results:\n{web_result[:500]}...")  # Show first 500 chars
except Exception as e:
    print(f"❌ game_web_search failed: {e}")

print("\n" + "="*60 + "\n")

Testing game_web_search tool:
--------------------------------------------------
✅ game_web_search executed successfully
Results:
1. Updates from the Pokémon Day 2024 Pokémon Presents
No Snippet
https://www.pokemon.com/us/pokemon-news/updates-from-the-pokemon-day-2024-pokemon-presents

2. No Mainline Game in 2024 : r/pokemon - Reddit
No Snippet
https://www.reddit.com/r/pokemon/comments/1b1efxk/no_mainline_game_in_2024/

3. New Pokémon Legends Announced for 2025 - IGN
No Snippet
https://www.ign.com/articles/new-pokemon-game-pokemon-presents-2024

4. Upcoming Pokémon Games - Release Dates, All Announced ...
No Snippet
https:...


